
<center>
  <img src="https://jupyter.org/assets/nav_logo.svg" alt="Jupyter" height="200" width="320"/>
</center>

[Jupyter Notebook](https://jupyter.org/) is a commonly used open-source 
tool in data science. It allows for organizing and sharing data processing 
and analysis code in a readable notebook style format. There are many 
[great examples](https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks) 
of Jupyter Notebook on the web. This blog post is part one of a three part series aimed at 
showing how this powerful tool can be used to analyze ROS data. Hopefully it 
will serve as a complete example of how to create your own notebook to 
analyze your robot's data. In part one we will use ROS and Gazebo to generate 
some simulation data that we will analyze in parts two and three.

## Getting Some Sample Robot Data
Before we begin our analysis of ROS data with Jupyter, we need some robot data. 
To get this data, we will use the simulator for the open-source robot, 
Bobble-Bot. This project contains a high fidelity simulation of a two 
wheeled self-balancing robot. The simulator has been validated against 
test runs of the real robot created by the engineers at [SOE](https://so.engineering). 
For more information, see the project's
[GitHub page](https://github.com/super-owesome/bobble_controllers). By the end of this 
post, we will have grabbed data from some simulation runs like the one shown below.

![BobbleSim](https://media.giphy.com/media/5QPazHiJQLuBGeiu5z/giphy.gif "Bobble-Bot Sim")

If you would like to focus on the ROS/Jupyter analysis aspect, you can skip the 
simulation steps below and proceed to download the ROS simulation data from [here](https://github.com/super-owesome/bobble_controllers/raw/master/analysis/notebooks/RosJupyterAnalysis/data/sample_data.zip) and jump ahead to [part 2]({static}/ros-analysis-part-2).

## Building and running the simulation
The Bobble-Bot simulator requires ROS and Gazebo. Follow the instructions 
[here](http://wiki.ros.org/melodic/Installation/Ubuntu)
and install ROS Melodic Desktop. Other recent versions of ROS should also work, 
but they are not officially  supported at this time. The simulator also makes 
use of the [Hector Gazebo](http://wiki.ros.org/hector_gazebo_plugins) 
plugins. Those can be installed using the command below.

```bash
apt-get install ros-melodic-hector-gazebo-plugins
```

Before starting the build process, make sure your ROS environment is active.

```bash
source /opt/ros/melodic/setup.bash
```

Get the simulation code and build it using catkin.

```bash
mkdir -p ~/bobble_workspace/src
cd ~/bobble_workspace/src
catkin_init_workspace
git clone https://github.com/super-owesome/bobble_controllers.git
git clone https://github.com/super-owesome/bobble_description.git
```
By now your bobble_workspace directory should look like so:

```text
└── src
    ├── bobble_controllers
    │   ├── analysis
    │   ├── config
    │   ├── launch
    │   ├── src
    |   ...
    ├── bobble_description
    │   ├── imgs
    │   ├── launch
    │   ├── meshes
    │   └── xacro
    |   ...
    └── CMakeLists.txt
```

Build the Bobble-Bot simulation by running this command:

```bash
catkin_make
catkin_make install
source install/setup.bash
```

The Bobble-Bot controller package comes with a set of launch files that can 
be used to generate data from different runs of the simulation. As an 
example, the apply_impulse_force.launch file can be used to run the Bobble-Bot 
simulation and apply an impulse force in the X direction. This launch file accepts 
a few launch parameters. 
Most notably, the X component of the impulse force, and a flag to enable/disable 
the 3D graphics. 

```bash
roslaunch bobble_controllers apply_impulse_force.launch \
impulse_force:=-1000 gui:=true
```

The launch file also instructs the simulation to log simulation data during the run.
The data is placed in the following location by default:

```bash
ls ~/.ros | grep impulse_data
impulse_data.bag
```

You can override this default location like so:

```bash
roslaunch bobble_controllers apply_impulse_force.launch \
impulse_force:=-1000 gui:=true out_file:=/path/to/file/name_no_ext
```

## Generating some more data
The bobble_controllers package comes with a set of automated tests that can be used 
to quickly and easily generate data from four different test runs of the 
Bobble-Bot balance controller. These tests can be run using the following 
commands.

```bash
cd ~/bobble_workspace
catkin run_tests -j1
```

These tests will take a few minutes to run. A lot is happening behind the scenes 
and your data is being generated. Now would be a good time to grab a cup 
of coffee. Once the tests have completed you can get a summary of the results using the 
commands below:

```bash
catkin_test_results --verbose .
Summary: 18 tests, 0 errors, 0 failures, 0 skipped
```

If all went well, your tests will have all passed. If you encounter a failure, 
post a comment below. The tests generated simulation results from four different 
runs and recorded the data from each into ROS bag files. The table below briefly 
summarizes each test that was performed.


| Test Name           | Description                                  | Source Code    |
| ------------------- | -------------------------------------------- | -------------- |
| No Balance          | Bobble-Bot uncontrolled                      | [no_balance.test](https://github.com/super-owesome/bobble_controllers/blob/master/test/no_balance.test) |
| Impulse force       | Balance control active, impulse applied.     | [impulse_force](https://github.com/super-owesome/bobble_controllers/blob/master/test/impulse_force.test) |
| Balance Control     | Balance control active, drive around.        | [balance_control](https://github.com/super-owesome/bobble_controllers/blob/master/test/balance_controller.test) |
| Drive square        | Balance control active, drive in square.     | [drive_square](https://github.com/super-owesome/bobble_controllers/blob/master/test/js_drive_square.test) |


The data can be found in the following directory:

```text
src/bobble_controllers/
└── test
    ├── data
    │   ├── balance.bag
    │   ├── balance.csv
    │   ├── drive_square_js_response.bag
    │   ├── drive_square_js_response.csv
    │   ├── impulse_force.bag
    │   ├── impulse_force.csv
    │   ├── no_balance.bag
    │   └── no_balance.csv
```

If anything went wrong, you can simply download the data [here](https://github.com/super-owesome/bobble_controllers/raw/master/analysis/notebooks/RosJupyterAnalysis/data/sample_data.zip).
Now that we have our data, we can move on to part two of this series where we will 
set up Jupyter Notebook to begin the analysis.